In [9]:
import cv2
import matplotlib.image as mplimg
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from pylab import rcParams
from pylab import plot, ginput, show, axis, imshow, draw
rcParams['figure.figsize'] = (6,6)

%matplotlib qt

In [25]:
imgPath = '/Users/kangyang/Downloads/image_rectification-master/images/floor.png'
im = mplimg.imread(imgPath)
plt.imshow(im)
plt.show()

In [11]:
def replotAffine(im,limits,lines=[[],[]],x=[[],[]],y=[[],[]],vPts=[]):
    # -- Settings for this function ---
    plot_lines  = True
    plot_vpts   = True
    plot_points = True
    plot_vline  = True
    # ---------------------------------
    if len(x) != len(y):
        raise Exception("len(x): %d, len(y): %d!" % (len(x), len(y)))
    if len(lines) != len(x):
        raise Exception("len(x): %d, len(lines): %d!" % (len(x), len(lines)))
    if len(vPts) != min(len(x[0]), len(x[1])):
        raise Exception("len(x[0]): %d, len(x[1]): %d, len(vpts): %d!" % (len(x[0]), len(x[1]), len(vPts)))

    #plt.close() # ginput does not allow new points to be plotted
    plt.imshow(im,cmap='gray')
    plt.show()
    plt.axis('image')
    ax = plt.gca()
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()

    # Determine how many lines to plot in red, leaving the last in green if the second needs to be picked
    nl1 = len(y[0])
    nl2 = len(y[1])
    if nl1 == nl2:
        nred = nl1
    else:
        nred = nl1 - 1
    # Plot extension of user-selected lines (dashed)
    if plot_lines:
        for k in range(nred):
            xx,yy = getPlotBoundsLine(limits, lines[0][k])
            plot(xx,yy,'r--')
        if nl1 - nred > 0:
            xx,yy = getPlotBoundsLine(limits, lines[0][nl1-1])
            plot(xx,yy,'g--')
        for l in lines[1]:
            xx,yy = getPlotBoundsLine(limits, l)
            plot(xx,yy,'b--')

    # Plot user-selected line segments (solid)
    if plot_points:
        # Plot lines: direction 1, all red but the last one green
        for k in range(0,nred):
            plot(x[0][k],y[0][k],'r-')
        if nl1 - nred > 0:
            plot(x[0][nl1-1],y[0][nl1-1],'g-')
        # Plot lines: direction 2
        for k in range(0,len(y[1])):
            plot(x[1][k],y[1][k],'b-')

    # Compute normalized vanishing points for plotting
    vPts_n = [[0,0] for x in vPts]
    vPtInImage = [True for x in vPts]
    for i in range(len(vPts)):
        if vPts[i][2] == 0:
            vPtInImage[i] = False
        else:
            vPts_n[i][0] = vPts[i][0] / vPts[i][2]
            vPts_n[i][1] = vPts[i][1] / vPts[i][2]
            vPtInImage[i] = vPts_n[i][0] < limits[0] and vPts_n[i][0] > 0 and vPts_n[i][1] < limits[1] and vPts_n[i][1] > 0

    # Plot vanishing points 
    if plot_vpts:
        for i in range(len(vPts_n)):
            if vPtInImage[i]:
                plot(vPts_n[i][0], vPts_n[i][1], 'yo')
    # Plot vanishing line
    if plot_vline:
        if len(vPts) == 2 and vPtInImage[0] and vPtInImage[1]:
            vLine = np.linalg.cross(vPts[0], vPts[1])
            xx,yy = getPlotBoundsLine(limits, vLine)
            plot(xx,yy,'y-')
            #plot([vPts_n[0][0],vPts_n[1][0]], [vPts_n[0][1],vPts_n[1][1]], 'y-')

    # Limit axes to the image
    plt.xlim([xmin,xmax])
    plt.ylim([ymin,ymax])
    

In [12]:
fig2 = plt.figure()
replotAffine(im, im.shape)

In [13]:
def getLine():
    # get mouse clicks
    pts = []
    while len(pts) == 0: # FIXME
        pts = ginput(n=2)
    pts_h = [[x[0],x[1],1] for x in pts]
    line = np.cross(pts_h[0], pts_h[1]) # line is [p0 p1 1] x [q0 q1 1]
    # return points that were clicked on for plotting
    x1=map(lambda x: x[0],pts) # map applies the function passed as 
    y1=map(lambda x: x[1],pts) # first parameter to each element of pts
    return x1,y1,line

In [14]:
lines = [[],[]]
x = [[],[]]
y = [[],[]]
vPts = []
nLinePairs = 2

# Get line pairs interactively
for i in range(0,2*nLinePairs):
    ii = i % 2
    if ii == 1:
        plt.suptitle('Click two points intersecting a line parallel to the green line')
    else:
        if i == 0:
            plt.suptitle('Click two points intersecting the first of two parallel lines')
        else:
            plt.suptitle('Click two points intersecting the first of two parallel lines not parallel to the first set')
    x1,y1,line = getLine()
    print line
    x[ii].append(x1)
    y[ii].append(y1)
    lines[ii].append(line)
    if ii == 1:
        nlp = len(lines[0])
        vPt = np.cross(lines[0][nlp-1], lines[1][nlp-1])
        if vPt[2] != 0.:
            vPt[0] = vPt[0] / vPt[2]
            vPt[1] = vPt[1] / vPt[2]
            vPt[2] = vPt[2] / vPt[2]
        vPts.append(vPt)

[  -156.29032258    185.90322581 -19002.56815817]
[  -103.64516129    204.          18568.90894901]
[    90.48387097    241.83870968 -40858.21748179]
[    141.48387097     258.29032258 -112528.31581686]


In [17]:
def getPlotBoundsLine(size, l):
    l = l.flatten(1)
    L = 0
    R = 1
    T = 2
    B = 3
    Nx = size[1]
    Ny = size[0]
    # lines intersecting image edges
    lbd = [[] for x in range(4)]
    lbd[L] = np.array([1.0, 0.0, 0.0])
    lbd[R] = np.array([1.0, 0.0, -float(Nx)])
    lbd[T] = np.array([0.0, 1.0, 0.0])
    lbd[B] = np.array([0.0, 1.0, -float(Ny)])
    I = [np.cross(l, l2) for l2 in lbd]

    # return T/F if intersection point I is in the bounds of the image
    Ied = [] # List of (x,y) where (x,y) is an intersection of the line with the boundary
    for i in [L, R]:
        if I[i][2] != 0:
            In1 = I[i][1] / I[i][2]
            if In1 > 0 and In1 < Ny:
                Ied.append(I[i][0:2]/I[i][2])

    for i in [T, B]:
        if I[i][2] != 0:
            In0 = I[i][0] / I[i][2]
            if In0 > 0 and In0 < Nx:
                Ied.append(I[i][0:2]/I[i][2])

    assert(len(Ied) == 2 or len(Ied) == 0)
    xx = [Ied[x][0] for x in range(0,len(Ied))]
    yy = [Ied[x][1] for x in range(0,len(Ied))]

    return xx,yy


In [18]:
replotAffine(im,im.shape,lines,x,y,vPts)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  from ipykernel import kernelapp as app


In [20]:
print("Vanishing points:")
print(vPts[0])
print(vPts[1])
vLine = np.cross(vPts[0], vPts[1])
print("Vanishing line:")
print(vLine)
H = np.identity(3)
H[2,0] = vLine[0] / vLine[2]
H[2,1] = vLine[1] / vLine[2]

print("H:")
print(H)

Vanishing points:
[-580.92709234 -386.17250555    1.        ]
[  1.53620669e+03  -4.05822999e+02   1.00000000e+00]
Vanishing line:
[  1.96504937e+01   2.11713378e+03   8.28994362e+05]
H:
[[  1.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00]
 [  2.37040137e-05   2.55385788e-03   1.00000000e+00]]


In [21]:
def myApplyH(im, H):
    return cv2.warpPerspective(im, H, (im.shape[1], im.shape[0]))

In [22]:
imRect = myApplyH(im, H)

In [26]:
plt.figure()
plt.imshow(imRect)
plt.show()